In [1]:
@file:DependsOn("ro.jf.funds:user-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:reporting-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:fund-sdk:1.0.0")
@file:DependsOn("com.charleskorn.kaml:kaml-jvm:0.67.0")
@file:DependsOn("ro.jf.funds:funds-notebook-client:1.0.0")
%use dataframe
%use kandy

import ro.jf.funds.client.notebook.*

val client = FundsClient()
val user = client.ensureUserExists("Johann-16.2")
user

UserTO(id=87036671-4f43-4d31-a5ec-971af7d96818, username=Johann-16.2)

In [2]:
import ro.jf.funds.user.sdk.UserSdk
import kotlinx.coroutines.runBlocking

val REPORT_DATA_CONFIGURATION_YAML_FILE = "../../data/provision/expenses-report-data-configuration.yaml"

In [3]:
import ro.jf.funds.fund.sdk.FundSdk
import ro.jf.funds.fund.api.model.FundName

val fundSdk = FundSdk()
val expenseFund = runBlocking { fundSdk.getFundByName(user.id, FundName("Expenses")) }
expenseFund

FundTO(id=316b24d1-5052-413e-b25a-068bb059a358, name=Expenses)

In [4]:
import ro.jf.funds.reporting.api.model.*
import kotlinx.serialization.json.Json
import com.charleskorn.kaml.Yaml

fun getReportDataConfiguration(): ReportDataConfigurationTO {
    val dataConfigurationRawJson: String = File(REPORT_DATA_CONFIGURATION_YAML_FILE).readText()
    val dataConfiguration = Yaml.default.decodeFromString(ReportDataConfigurationTO.serializer(), dataConfigurationRawJson)
    return dataConfiguration
}
val dataConfiguration = getReportDataConfiguration()
dataConfiguration

ReportDataConfigurationTO(currency=Currency(value=RON), filter=RecordFilterTO(labels=[basic, home, shopping_services, transport, fun, gifts, development]), groups=[ReportGroupTO(name=basic, filter=RecordFilterTO(labels=[basic])), ReportGroupTO(name=home, filter=RecordFilterTO(labels=[home])), ReportGroupTO(name=shopping_services, filter=RecordFilterTO(labels=[shopping_services])), ReportGroupTO(name=transport, filter=RecordFilterTO(labels=[transport])), ReportGroupTO(name=fun, filter=RecordFilterTO(labels=[fun])), ReportGroupTO(name=gifts, filter=RecordFilterTO(labels=[gifts])), ReportGroupTO(name=development, filter=RecordFilterTO(labels=[development])), ReportGroupTO(name=investment, filter=RecordFilterTO(labels=[investment]))], features=ReportDataFeaturesConfigurationTO(net=NetReportFeatureTO(enabled=true, applyFilter=true), valueReport=GenericReportFeatureTO(enabled=true), groupedNet=GenericReportFeatureTO(enabled=true), groupedBudget=GroupedBudgetReportFeatureTO(enabled=true, dist

In [5]:
import ro.jf.funds.reporting.sdk.ReportingSdk
import kotlinx.coroutines.delay
import ro.jf.funds.commons.model.labelsOf
import ro.jf.funds.commons.model.Currency
import java.time.Instant

val reportingSdk = ReportingSdk()
val reportView = runBlocking {
    val reportViewName = "Expense report"
    val existingReportView = reportingSdk.listReportViews(user.id).items.firstOrNull { it.name == reportViewName }
    if (existingReportView != null) {
        return@runBlocking existingReportView
    }
    val request = CreateReportViewTO(reportViewName, expenseFund.id, dataConfiguration)
    var task: ReportViewTaskTO = reportingSdk.createReportView(user.id, request)
    val timeout = Instant.now().plusSeconds(120)
    while (task is ReportViewTaskTO.InProgress && Instant.now().isBefore(timeout)) {
        delay(2000)
        task = reportingSdk.getReportViewTask(user.id, task.taskId)
    }
    if (task is ReportViewTaskTO.Completed) {
        task.report
    } else if (task is ReportViewTaskTO.Failed) {
        throw IllegalStateException("Report view creation failed on task $task")
    } else {
        throw IllegalStateException("Report view creation timed out on task $task")
    }
}
reportView

ReportViewTO(id=328052a4-8d2e-41c5-920f-4013c678390b, name=Expense report, fundId=316b24d1-5052-413e-b25a-068bb059a358, dataConfiguration=ReportDataConfigurationTO(currency=Currency(value=RON), filter=RecordFilterTO(labels=[basic, home, shopping_services, transport, fun, gifts, development]), groups=null, features=ReportDataFeaturesConfigurationTO(net=NetReportFeatureTO(enabled=true, applyFilter=true), valueReport=GenericReportFeatureTO(enabled=true), groupedNet=GenericReportFeatureTO(enabled=false), groupedBudget=GroupedBudgetReportFeatureTO(enabled=false, distributions=[]))))

In [6]:
import ro.jf.funds.reporting.api.model.*

val granularInterval = GranularDateInterval(
    interval = DateInterval(
        LocalDate.parse("2019-01-01"),
        LocalDate.parse("2020-06-30")
    ),
    granularity = TimeGranularity.MONTHLY
)
val report = runBlocking {
    reportingSdk.getReportViewData(user.id, reportView.id, granularInterval)
}
report

ReportDataTO(viewId=328052a4-8d2e-41c5-920f-4013c678390b, granularInterval=GranularDateInterval(interval=DateInterval(from=2019-01-01, to=2020-06-30), granularity=MONTHLY), data=[ReportDataItemTO(timeBucket=DateInterval(from=2019-01-01, to=2019-01-31), net=-4468.51, value=ValueReportTO(start=0.0, end=2481.52, min=0.0, max=0.0), groups=[ReportDataGroupItemTO(group=basic, net=-2305.04, allocated=2085.009, left=-220.03157528660788), ReportDataGroupItemTO(group=home, net=-500.0, allocated=2085.009, left=1585.0083652529643), ReportDataGroupItemTO(group=shopping_services, net=-385.57, allocated=347.5015, left=-38.06850298024303), ReportDataGroupItemTO(group=transport, net=-204.24, allocated=347.5015, left=143.2615091555513), ReportDataGroupItemTO(group=fun, net=-1031.16, allocated=1042.5045, left=11.344048161923961), ReportDataGroupItemTO(group=gifts, net=0.0, allocated=834.0036, left=834.0036081939585), ReportDataGroupItemTO(group=development, net=-42.5, allocated=208.5009, left=166.0009530

In [7]:
import java.math.BigDecimal
import org.jetbrains.kotlinx.kandy.dsl.* // For creating visualizations
import org.jetbrains.letsPlot.scale.scaleYContinuous

data class MonthlyExpense(val month: LocalDate, val net: Double)

val monthlyReportDF = report.data
    .mapNotNull { reportDataItem ->
        reportDataItem.net?.let { amount ->
        MonthlyExpense(reportDataItem.timeBucket.from, amount.negate().toDouble())
            }
    }
    .toDataFrame()

plot(monthlyReportDF) {
    x("month") {

    }
    bars {
        y("net")

        fillColor = Color.ORANGE
    }
    layout {
        title = "Monthly expenses"
        size = 1800 to 1000
    }
    line {
        y.constant(0)
    }
    line {
        y.constant(10000)
    }
    line {
        y.constant(20000)
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="msZCja"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Monthly expenses"
},
"mapping":{
},
"data":{
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12],
"net":[4468.51,4301.32,9171.45,6967.76,9043.57,7024.09,11232.58,25034.218092,4537.4484992,6776.5,9787.68,5529.22,4702.18,5442.03,35747.94,17363.44,0.0,0.0]
},
"ggsize":{
"width":1800.0,
"height":1000.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"month",
"y":"net"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"dodge",
"geom":"bar",
"fill":"#fc8452",
"data":{
}
},{
"mapping":{
"x":"month"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month"
},
"stat":"identity",
"sampling":"none",
"y":10000.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month"
},
"stat":"identity",
"sampling":"none",
"y":20000.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"datetime",
"column":"month"
},{
"type":"float",
"column":"net"
}]
},
"spec_id":"2"
};
 var containerDiv = document.getElementById("msZCja");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 1800.0,
 height: 1000.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M114.05412710276869 921.0 L114.05412710276869 921.0 L203.09159726610233 921.0 L283.51253805878514 921.0 L372.5500082221115 921.0 L458.71530192856153 921.0 L547.7527720918879 921.0 L633.9180657983379 921.0 L722.9555359616643 921.0 L811.9930061249979 921.0 L898.1582998314407 921.0 L987.1957699947743 921.0 L1073.361063701217 921.0 L1162.3985338645507 921.0 L1251.436004027877 921.0 L1334.7291212774435 921.0 L1423.7665914407698 921.0 L1509.9318851472199 921.0 L1598.9693553105535 921.0 " fill="none" stroke-width="1.6500000000000001" stroke="rgb(71,71,71)" stroke-opacity="1.0">
 
 
 
 
 
 <path d="M114.05412710276869 675.6312159126212 L114.05412710276869 675.6312159126212 L203.09159726610233 675.6312159126212 L283.51253805878514 675.6312159126212 L372.5500082221115 675.6312159126212 L458.71530192856153 675.6312159126212 L547.7527720918879 675.6312159126212 L633.9180657983379 675.6312159126212 L722.9555359616643 675.6312159126212 L811.9930061249979 675.6312159126212 L898.1582998314407 675.6312159126212 L987.1957699947743 675.6312159126212 L1073.361063701217 675.6312159126212 L1162.3985338645507 675.6312159126212 L1251.436004027877 675.6312159126212 L1334.7291212774435 675.

In [8]:
val categories = mapOf(
    "basic" to Color.GREEN,
    "home" to Color.PEACH,
    "shopping_services" to Color.LIGHT_GREEN,
    "transport" to Color.BLUE,
    "fun" to Color.ORANGE,
    "gifts" to Color.RED,
    "development" to Color.PURPLE,
)

fun getGroupedExpensesDataFrame(): DataFrame<*> {
    val categoriesNetExtractor: Map<String, (List<ReportDataGroupItemTO>) -> Double> =
        categories
            .keys
            .map { group ->
                group to { groups: List<ReportDataGroupItemTO> ->
                    groups.first { it.group == group }.net?.negate()?.toDouble() ?: 0.0
                }
            }
            .toMap()

    return report.data
        .map { monthlyData ->
            monthlyData.timeBucket.from to categoriesNetExtractor.mapValues { (_, extractor) -> extractor(monthlyData.groups!!) }
        }
        .let {
            dataFrameOf(
                "month" to it.flatMap { (month, values) -> values.map { month } },
                "value" to it.flatMap { (_, values) -> values.map { it.value } },
                "category" to it.flatMap { (_, values) -> values.keys }
            )
        }
}
getGroupedExpensesDataFrame().groupBy("category").plot {
    x("month")
    line {
        y("value")
        color("category") {
            scale = categorical(
                *categories.map { (category, color) -> category to color }.toTypedArray()
            )
        }
    }
    line {
        y.constant(0)
    }
    layout {
        title = "Expenses by category"
        size = 1200 to 600
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="C2ZcZQ"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Expenses by category"
},
"mapping":{
},
"data":{
"&merged_groups":["basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","home","home","home","home","home","home","home","home","home","home","home","home","home","home","home","home","home","home","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","development","development","development","development","development","development","development","development","development","development","development","development","development","development","development","development","development","development"],
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12],
"category":["basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","home","home","home","home","home","home","home","home","home","home","home","home","home","home","home","home","home","home","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shoppin

In [10]:
data class MonthlyExpense(val month: LocalDate, val start: Double, val end: Double)

val df = report.data
    .mapNotNull { item ->
        item.value?.let { valueReport ->
            MonthlyExpense(
                item.timeBucket.from,
                valueReport.start.toDouble(),
                valueReport.end.toDouble()
            )
        }
    }
    .let {
        dataFrameOf(
            "month" to it.map { it.month }.let { it + it },
            "value" to it.map { it.start } + it.map { it.end },
            "type" to List(it.size) { "startValue" } + List(it.size) { "endValue" }
        )
    }

df.groupBy("type").plot {
    x("month")
    line {
        y("value")
        color("type") {
            scale = categorical("startValue" to Color.ORANGE, "endValue" to Color.BLUE)
        }
    }
    line {
        y.constant(0)
    }
    layout {
        title = "Expense fund value"
        size = 1200 to 600
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="8uSRIL"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Expense fund value"
},
"mapping":{
},
"data":{
"&merged_groups":["startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue"],
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12],
"type":["startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue"],
"value":[0.0,2481.52,5185.08,11301.26,11528.2727350348,10643.764995302,11871.901585538,8638.3980216284,-6550.7530315744,-873.7461008179,4052.2818580708,6037.1321957254,12396.0035787372,18628.0469211958,24175.7814668309,22046.8575483166,24076.3746926345,24072.5638477149,2481.52,5185.08,11301.26,11526.0160226864,10643.310981436,11868.2374648476,8637.9027559568,-6550.8393119068,-872.2869408013,4051.7634872547,6037.1198616282,12394.3761693426,18628.6707953371,24174.7828379781,22045.4085328505,24075.5881809199,24072.6945814626,24076.192806606]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"values":["#fc8452","#5470c6"],
"limits":["startValue","endValue"]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"month",
"y":"value",
"color":"type",
"group":"&merged_groups"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"group":"&merged_groups"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"type"
},{
"type":"datetime",
"column":"month"
},{
"type":"float",
"column":"value"
},{
"type":"str",
"column":"&merged_groups"
}]
},
"spec_id":"8"
};
 var containerDiv = document.getElementById("8uSRIL");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 1200.0,
 height: 600.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M45.13672835783291 396.3400842655638 L45.13672835783291 396.3400842655638 L99.

In [11]:
val dimensions = mapOf(
    "spent" to Color.RED,
    "allocated" to Color.GREEN,
    "left" to Color.YELLOW,
)
val group = "basic"
val df = dimensions
    .map { dimension ->
        dataFrameOf(
            "month" to report.data.map { it.timeBucket.from },
            "type" to List(report.data.size) { dimension.key },
            "value" to report.data
                .map { it.groups!!.single { it.group == group } }
                .map { groupData ->
                    when (dimension.key) {
                        "spent" -> groupData.net!!.times(BigDecimal(-1))
                        "allocated" -> groupData.allocated
                        "left" -> groupData.left
                        else -> error("unexpected key")
                    }
                }
        )
    }
    .reduce { a, b -> a.concat(b) }

df.groupBy("type").plot {
    x("month")
    line {
        y("value")
        color("type") {
            scale = categorical(
                *dimensions.map { (category, color) -> category to color }.toTypedArray()
            )
        }
    }
    line {
        y.constant(0)
    }
    layout {
        title = "Basic expenses"
        size = 1200 to 600
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="3LzOxL"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Basic expenses"
},
"mapping":{
},
"data":{
"&merged_groups":["spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left"],
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12],
"type":["spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left"],
"value":[2305.04,2509.31,3347.92,2293.91,2108.18,2382.69,3709.61,2493.78,1587.87,2253.23,2992.86,2517.62,2158.1,2165.51,2324.86,948.25,0.0,0.0,2085.009,2101.464,4586.289,2013.904486352192,2286.2532,2311.6492,2159.9541,2658.6387,2759.141749714605,2925.985,2945.58,2971.8825,2514.8522,2529.8965,7733.9823,4073.0592,0.0,0.0,-220.03157528660788,-627.8762351468594,610.4954125346916,331.15879866505435,509.73832654819284,438.7774233960152,-1100.4104621696342,-933.4779202388673,238.81001665201296,912.7250529664104,864.1482624237958,1318.0777079662928,1674.9548958573187,2037.4991072100868,7440.434833714446,10565.858473023773,10564.588550735458,10566.123826831788]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"values":["#ee6666","#3ba272","#fac858"],
"limits":["spent","allocated","left"]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"month",
"y":"value",
"color":"type",
"group":"&merged_groups"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"group":"&merged_groups"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"type"
},{
"type":"datetime",
"column":"month"
},{
"type":"str",
"column":"&merged_groups"
}]
},
"spec_id":"11"
};
 var containerDiv = document.getElementById("3LzOxL");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 1200.0,
 height:

In [57]:
import org.jetbrains.kotlinx.kandy.ir.Plot

val dimensions = mapOf(
    "spent" to Color.RED,
    "allocated" to Color.GREEN,
    "left" to Color.YELLOW,
)

fun plotGroupData(plotTitle: String, extractor: (List<ReportDataGroupItemTO>, String) -> BigDecimal): Plot =
    dimensions
        .map { dimension ->
            dataFrameOf(
                "month" to report.data.map<ReportDataItemTO, LocalDate> { it.timeBucket.from },
                "type" to List<String>(report.data.size) { dimension.key },
                "value" to report.data.map { extractor.invoke(it.groups!!, dimension.key) }
            )
        }
        .reduce<DataFrame<*>, DataFrame<*>> { a, b -> a.concat<Any?>(b) }
        .groupBy("type")
        .plot {
            x("month")
            line {
                y.constant(0)
            }
            area {
                y("value")
                borderLine {
                    color("type") {
                        scale = categorical(
                            *dimensions.map { (category, color) -> category to color }.toTypedArray()
                        )
                    }
                }
            }
            layout {
                title = plotTitle
                size = 1200 to 600
            }
        }

fun plotGroup(title: String, group: String): Plot =
    plotGroupData(title) { groupedData: List<ReportDataGroupItemTO>, dimensionKey: String ->
        groupedData
            .single { it.group == group }
            .let { groupData ->
                when (dimensionKey) {
                    "spent" -> groupData.net!!.times(BigDecimal(-1))
                    "allocated" -> groupData.allocated!!
                    "left" -> groupData.left!!
                    else -> error("unexpected key")
                }
            }
    }

fun plotGroupsTotal(): Plot =
    plotGroupData("Total expenses") { groupedData: List<ReportDataGroupItemTO>, dimensionKey: String ->
        groupedData
            .map { groupData ->
                when (dimensionKey) {
                    "spent" -> groupData.net!!.times(BigDecimal(-1))
                    "allocated" -> groupData.allocated!!
                    "left" -> groupData.left!!
                    else -> error("unexpected key")
                }
            }
            .fold(BigDecimal.ZERO) { acc, value -> acc + value }
    }



In [58]:
plotGroupsTotal()

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="SUKcCm"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Total expenses"
},
"mapping":{
},
"data":{
"&merged_groups":["spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left"],
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12],
"type":["spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left"],
"value":[4468.51,4301.32,9171.45,6967.76,9043.57,7024.09,11232.58,25034.218092,4537.4484992,6776.5,9787.68,5529.22,4702.18,5442.03,35747.94,17363.44,0.0,0.0,6950.03,7004.88,15287.63,7192.5160226864,8165.19,8255.89,7999.83,9846.81,10219.0435174615,11703.94,11782.32,11887.53,10934.14,10999.55,33626.01,19395.52,0.0,0.0,2481.5184055566156,5185.078542790578,11301.25672763291,11526.01237885718,10643.304704710494,11868.181672377845,8637.860072199115,-6550.901207576708,-872.3493673543176,4051.699300458543,6037.049349382055,12394.297585419017,18628.555462398985,24174.681741944973,22045.308352509317,24075.49539009924,24072.601729515376,24076.100028557907]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"values":["#ee6666","#3ba272","#fac858"],
"limits":["spent","allocated","left"]
}],
"layers":[{
"mapping":{
"x":"month",
"group":"&merged_groups"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"value",
"color":"type",
"group":"&merged_groups"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"area",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"type"
},{
"type":"datetime",
"column":"month"
},{
"type":"str",
"column":"&merged_groups"
}]
},
"spec_id":"62"
};
 var containerDiv = document.getElementById("SUKcCm");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 1200.0,
 height: 600.0


In [59]:
plotGroup("Basic Expenses", "basic")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="1wi3oN"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Basic Expenses"
},
"mapping":{
},
"data":{
"&merged_groups":["spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left"],
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12],
"type":["spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left"],
"value":[2305.04,2509.31,3347.92,2293.91,2108.18,2382.69,3709.61,2493.78,1587.87,2253.23,2992.86,2517.62,2158.1,2165.51,2324.86,948.25,0.0,0.0,2085.009,2101.464,4586.289,2013.904486352192,2286.2532,2311.6492,2159.9541,2658.6387,2759.141749714605,2925.985,2945.58,2971.8825,2514.8522,2529.8965,7733.9823,4073.0592,0.0,0.0,-220.03157528660788,-627.8762351468594,610.4954125346916,331.15879866505435,509.73832654819284,438.7774233960152,-1100.4104621696342,-933.4779202388673,238.81001665201296,912.7250529664104,864.1482624237958,1318.0777079662928,1674.9548958573187,2037.4991072100868,7440.434833714446,10565.858473023773,10564.588550735458,10566.123826831788]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"values":["#ee6666","#3ba272","#fac858"],
"limits":["spent","allocated","left"]
}],
"layers":[{
"mapping":{
"x":"month",
"group":"&merged_groups"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"value",
"color":"type",
"group":"&merged_groups"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"area",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"type"
},{
"type":"datetime",
"column":"month"
},{
"type":"str",
"column":"&merged_groups"
}]
},
"spec_id":"65"
};
 var containerDiv = document.getElementById("1wi3oN");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 1200.0,
 height:

In [60]:
plotGroup("Home Expenses", "home")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="ReSkXH"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Home Expenses"
},
"mapping":{
},
"data":{
"&merged_groups":["spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left"],
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12],
"type":["spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left"],
"value":[500.0,609.16,1763.9,400.0,400.0,467.23,420.0,18881.0,423.24,456.78,400.0,400.0,400.0,2736.62,29698.21,16136.26,0.0,0.0,2085.009,2101.464,4586.289,2301.605127259648,2612.8608,2641.8848,2559.9456,3150.9792,3270.09392558768,3979.3396,4005.9888,4041.7602,3936.2904,3959.838,12105.3636,7370.2976,0.0,0.0,1585.0083652529643,3077.311115282201,5899.700682186836,7800.107154391456,10008.067335809888,12174.231586340718,14278.233000077309,-1461.7268978674037,1391.5041224510844,4914.558979593785,8487.4836063041,12130.283567178312,15669.33305011951,16886.562948830328,-691.0838188749666,-9460.97862698152,-9459.84150143149,-9461.216232540275]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"values":["#ee6666","#3ba272","#fac858"],
"limits":["spent","allocated","left"]
}],
"layers":[{
"mapping":{
"x":"month",
"group":"&merged_groups"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"value",
"color":"type",
"group":"&merged_groups"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"area",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"type"
},{
"type":"datetime",
"column":"month"
},{
"type":"str",
"column":"&merged_groups"
}]
},
"spec_id":"68"
};
 var containerDiv = document.getElementById("ReSkXH");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 1200.0,
 height: 600.0
 }
 };
 var f

In [61]:
plotGroup("Transport Expenses", "transport")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="X1N81g"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Transport Expenses"
},
"mapping":{
},
"data":{
"&merged_groups":["spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left"],
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12],
"type":["spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left"],
"value":[204.24,207.85,1214.12,281.75,554.64,348.18,288.04,1474.32,331.47,238.42,2502.33,2020.41,608.51,161.2,230.49,0.0,0.0,0.0,347.5015,350.244,764.3815,503.476121588048,571.5633,577.9123,719.9847,886.2129,919.713916571535,1287.4334,1296.0552,1307.6283,1312.0968,1319.946,4035.1212,2715.3728,0.0,0.0,143.2615091555513,285.65537905381814,-164.08465638463045,57.84717692625068,74.97044542327076,304.85265129291395,737.128485662555,150.641256250893,742.7042454341881,1792.6337883705194,583.3919760756717,-129.42447143309403,574.0337205782953,1731.6676443108029,5530.0060557743855,8245.926907586352,8244.935820454673,8246.133998009826]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"values":["#ee6666","#3ba272","#fac858"],
"limits":["spent","allocated","left"]
}],
"layers":[{
"mapping":{
"x":"month",
"group":"&merged_groups"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"value",
"color":"type",
"group":"&merged_groups"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"area",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"type"
},{
"type":"datetime",
"column":"month"
},{
"type":"str",
"column":"&merged_groups"
}]
},
"spec_id":"71"
};
 var containerDiv = document.getElementById("X1N81g");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 1200.0,
 height: 600.0
 }
 };
 var fig 

In [62]:
plotGroup("Shopping & Services Expenses", "shopping_services")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="DbwFVF"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Shopping & Services Expenses"
},
"mapping":{
},
"data":{
"&merged_groups":["spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left"],
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12],
"type":["spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left"],
"value":[385.57,0.0,609.77,1481.1,936.99,2494.99,463.18,0.0,82.99,342.57,0.0,0.0,149.99,0.0,1259.78,0.0,0.0,0.0,347.5015,350.244,764.3815,431.550961361184,489.9114,495.3534,639.9864,787.7448,817.52348139692,936.3152,942.5856,951.0024,656.0484,659.973,2017.5606,969.776,0.0,0.0,-38.06850298024303,312.17560556941356,466.78669783868344,-582.4583358325144,-1028.7030820166156,-3026.535949872354,-2838.1799866567903,-2047.7653084781705,-1318.5766095678846,-728.3829525280061,217.1049889380047,1167.9370367558117,1674.2419810397728,2333.1895334739042,3085.1306650513934,4055.3028860002164,4054.8154746333175,4055.404731966278]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"values":["#ee6666","#3ba272","#fac858"],
"limits":["spent","allocated","left"]
}],
"layers":[{
"mapping":{
"x":"month",
"group":"&merged_groups"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"value",
"color":"type",
"group":"&merged_groups"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"area",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"type"
},{
"type":"datetime",
"column":"month"
},{
"type":"str",
"column":"&merged_groups"
}]
},
"spec_id":"74"
};
 var containerDiv = document.getElementById("DbwFVF");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 1200.0,
 height: 600.0
 }
 };
 var fig = 

In [63]:
plotGroup("Fun Expenses", "fun")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="JOS0Ky"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Fun Expenses"
},
"mapping":{
},
"data":{
"&merged_groups":["spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left"],
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12],
"type":["spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left"],
"value":[1031.16,570.0,1237.89,2109.0,2235.76,0.0,790.09,1980.318092,1753.8784992,405.5,0.0,100.0,579.01,0.0,1287.38,0.0,0.0,0.0,1042.5045,1050.732,2293.1445,935.027082949232,1061.4747,1073.2657,879.9813,1083.1491,1124.094786920765,1053.3546,1060.4088,1069.8777,765.3898,769.9685,2353.8207,969.776,0.0,0.0,11.344048161923961,492.0763237223371,1547.3297861343049,373.96947761078286,-800.3903985837532,273.5632453084898,365.7616465020714,-531.9223301553407,-1167.5223831318137,-522.7167689068483,539.2587516958968,1509.0354018872004,1695.7229409145743,2464.5796460984207,3524.2445467760654,4494.5001062621295,4493.959907292558,4494.612982345163]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"values":["#ee6666","#3ba272","#fac858"],
"limits":["spent","allocated","left"]
}],
"layers":[{
"mapping":{
"x":"month",
"group":"&merged_groups"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"value",
"color":"type",
"group":"&merged_groups"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"area",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"type"
},{
"type":"datetime",
"column":"month"
},{
"type":"str",
"column":"&merged_groups"
}]
},
"spec_id":"77"
};
 var containerDiv = document.getElementById("JOS0Ky");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 1200.0,
 height: 600.0
 }
 };
 var 

In [65]:
plotGroup("Gifts Expenses", "gifts")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="VqwICQ"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Gifts Expenses"
},
"mapping":{
},
"data":{
"&merged_groups":["spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left"],
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12],
"type":["spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left"],
"value":[0.0,392.0,147.85,402.0,1930.0,1330.0,5561.66,204.8,351.0,20.0,3892.49,491.19,766.57,330.2,141.92,198.28,0.0,0.0,834.0036,840.5856,1834.5156,863.101922722368,979.8228,990.7068,879.9813,1083.1491,1124.094786920765,1170.394,1178.232,1188.753,1202.7554,1209.9505,3698.8611,2133.5072,0.0,0.0,834.0036081939585,1282.5894014257099,2969.2540355782603,3429.7234746152117,2478.5613934107714,2137.606395624506,-2531.8253836770614,-1651.000130651217,-881.6457300684935,265.6713352545599,-2439.3422707355917,-1742.615843643468,-1306.9651072542067,-427.5579888240904,3129.3623715529416,5064.771544765618,5064.162804237213,5064.898742798751]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"values":["#ee6666","#3ba272","#fac858"],
"limits":["spent","allocated","left"]
}],
"layers":[{
"mapping":{
"x":"month",
"group":"&merged_groups"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"value",
"color":"type",
"group":"&merged_groups"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"area",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"type"
},{
"type":"datetime",
"column":"month"
},{
"type":"str",
"column":"&merged_groups"
}]
},
"spec_id":"83"
};
 var containerDiv = document.getElementById("VqwICQ");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 1200.0,
 height: 600.0
 }
 };
 var fig = 

In [66]:
plotGroup("Development Expenses", "development")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="EsKBas"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Development Expenses"
},
"mapping":{
},
"data":{
"&merged_groups":["spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left"],
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12],
"type":["spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left"],
"value":[42.5,13.0,850.0,0.0,878.0,1.0,0.0,0.0,7.0,3060.0,0.0,0.0,40.0,48.5,805.3,80.65,0.0,0.0,208.5009,210.1464,458.6289,143.850320453728,163.3038,165.1178,159.9966,196.9362,204.38087034923,351.1182,353.4696,356.6259,546.707,549.9775,1681.3005,1163.7312,0.0,0.0,166.00095305906856,363.14695288395745,-28.225230255235743,115.66463248093945,-598.9393158812612,-434.3136797124448,-272.8472275393346,-75.64987643660173,122.37697087658877,-2582.7901342918776,-2214.995965319822,-1858.995813292037,-1352.7660188562795,-851.2591491544806,27.213698515050215,1110.1140994426719,1109.9806735936456,1110.141979146377]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"values":["#ee6666","#3ba272","#fac858"],
"limits":["spent","allocated","left"]
}],
"layers":[{
"mapping":{
"x":"month",
"group":"&merged_groups"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"value",
"color":"type",
"group":"&merged_groups"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"area",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"type"
},{
"type":"datetime",
"column":"month"
},{
"type":"str",
"column":"&merged_groups"
}]
},
"spec_id":"86"
};
 var containerDiv = document.getElementById("EsKBas");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 1200.0,
 height: 600.0
 }
 };
 var fig = LetsPlot.buildPlotFro

In [67]:
plotGroup("Investment Expenses", "investment")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="cuSJ1A"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Investment Expenses"
},
"mapping":{
},
"data":{
"&merged_groups":["spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left"],
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12],
"type":["spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left"],
"value":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"values":["#ee6666","#3ba272","#fac858"],
"limits":["spent","allocated","left"]
}],
"layers":[{
"mapping":{
"x":"month",
"group":"&merged_groups"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"value",
"color":"type",
"group":"&merged_groups"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"area",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"type"
},{
"type":"datetime",
"column":"month"
},{
"type":"str",
"column":"&merged_groups"
}]
},
"spec_id":"89"
};
 var containerDiv = document.getElementById("cuSJ1A");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 1200.0,
 height: 600.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M46.17771279455701 260.5 L46.17771279455701 260.5 L101.55524071